<h3>KMeans Implementation</h3>

In [3]:
import numpy as np
import sklearn
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix,classification_report

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()


In [4]:
x_train= x_train.reshape(len(x_train),-1)
x_test= x_test.reshape(len(x_test),-1)

In [5]:
x_train=x_train/255

In [6]:
clusters=KMeans(n_clusters=10).fit(x_train)

In [7]:
print(confusion_matrix(clusters.labels_,y_train))

[[   0    0    1    0    0  258    1   22    6 2615]
 [3404  236  115 1684  873    2 1053    0   22    2]
 [ 201 5413    9 3209  154    1   62    0   28    2]
 [ 167   63 3519   49 3596    0 1954    0  269    1]
 [  23    3   27    5   15    4   62    1 2203    4]
 [   0    0    0    0    0  474    0  763   61 2994]
 [  27    0   27    7   29   13   17    0 2449    0]
 [   2    0    1    0    0 1445    6 4703  238  177]
 [1583  129 1786  524 1081   29 2071    0  228   35]
 [ 593  156  515  522  252 3774  774  511  496  170]]


<h3>Relabeling </h3>

In [8]:
relabel = np.choose(clusters.labels_,[9,0,1,4,8,9,8,7,6,6])

In [9]:
print(confusion_matrix(relabel,y_train))

[[3404  236  115 1684  873    2 1053    0   22    2]
 [ 201 5413    9 3209  154    1   62    0   28    2]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [ 167   63 3519   49 3596    0 1954    0  269    1]
 [   0    0    0    0    0    0    0    0    0    0]
 [2176  285 2301 1046 1333 3803 2845  511  724  205]
 [   2    0    1    0    0 1445    6 4703  238  177]
 [  50    3   54   12   44   17   79    1 4652    4]
 [   0    0    1    0    0  732    1  785   67 5609]]


In [10]:
cf = confusion_matrix(relabel,y_train)

In [11]:
sum=0
for i in range(10):
    for j in range(10):
        if i==j:
            sum+=cf[i][j]

            

In [12]:
print((sum/60000)*100)

50.370000000000005


<h3>AutoEncoder with Kmeans clustering</h3>

In [ ]:
from keras.datasets import mnist
import numpy as np
import tensorflow as tf

(x_train, y_train), (x_test,y_test) = tf.keras.datasets.fashion_mnist.load_data()


In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1)) 
print(x_train.shape)
from sklearn.model_selection import train_test_split
train_X,valid_X,train_ground,valid_ground = train_test_split(x_train,x_train, test_size=0.2)


<h3>Autoencoder Implementation</h3>

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model

input_img = Input(shape=(28,28,1))
x = Conv2D(16,(3,3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(8,(3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(8,(3,3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2,2), padding='same', name='encoder')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')


In [ ]:
xx=autoencoder.fit(train_X, train_ground,
                epochs=150,
                batch_size=256,
                shuffle=True,
                validation_data=(valid_X, valid_ground))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(xx.history['loss'])
plt.plot(xx.history['val_loss'])
plt.title('Epoch vs Loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

In [ ]:
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('encoder').output)


In [ ]:
ab= encoder.predict(x_test)

In [ ]:
print(ab.shape)

In [ ]:
bc = np.reshape(ab,(10000,128))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
clusters=KMeans(n_clusters=10).fit(bc)

In [ ]:
predicted_label = clusters.predict(bc)

In [ ]:
print(confusion_matrix(predicted_label,y_test))

<h3>Relabeling Labels</h3>

In [ ]:
label_final = np.choose(predicted_label,[3,8,7,6,9,1,0,4,9,8])

In [ ]:
print(confusion_matrix(label_final,y_test))

In [ ]:
cf1= confusion_matrix(label_final,y_test)
sum=0
for i in range(10):
    for j in range(10):
        if i==j:
            sum+=cf1[i][j]

print((sum/10000)*100)

<h3>AutoEncoder with GMM Clustering</h3>

In [ ]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=10,covariance_type='full').fit(bc)
labels1 = gmm.predict(bc)


<h3>Relabeling Labels</h3>

In [ ]:
from scipy.stats import mode
import numpy as np

labels = np.zeros_like(labels1)
for i in range(10):
    mask = (labels1 == i)
    labels[mask] = mode(y_test[mask])[0]

In [ ]:
print(confusion_matrix(y_test,labels))

In [ ]:
cf2= confusion_matrix(y_test,labels)
sum=0
for i in range(10):
    for j in range(10):
        if i==j:
            sum+=cf2[i][j]

print((sum/10000)*100)
